# 1. Import packages and collect data

In [35]:
import tensorflow as tf
import string
import requests
import pandas as pd
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [36]:
response = requests.get('https://raw.githubusercontent.com/laxmimerit/poetry-data/master/adele.txt')

response

<Response [200]>

Let's analyze our training data

In [37]:
data = response.text.splitlines()

In [38]:
# number of lines in our data
len(data)

2400

In [39]:
#number of words in our data
len(" ".join(data))

91330

 # 2. Build LSTM model and prepare X,y datasets 

In [40]:
#tokenization

token = Tokenizer()
token.fit_on_texts(data)

In [41]:
encoded_text  = token.texts_to_sequences(data)

In [42]:
vocab_size  = len(token.word_counts) + 1
vocab_size #number of unique words in our data

1396

In [43]:
#preview our encoded texts data
for i in range(5):
  print(encoded_text[i])

[254, 21, 219, 725]
[117, 8, 80, 153, 3, 133]
[14, 10, 726, 727]
[41, 56, 2, 603, 3, 728, 1, 68, 517, 2, 40, 3, 518, 41]
[1, 23, 107, 189, 300, 9, 57]


# 3. Prepare the training data

In [44]:
datalist = []
for d  in encoded_text:
  if len(d) >= 2:
    for i in range(2,len(d)):
      datalist.append(d[:i])
      print(d[:i])


Streaming output truncated to the last 5000 lines.
[197, 203, 438, 16, 1167]
[27, 6]
[27, 6, 218]
[27, 6, 218, 77]
[27, 6, 218, 77, 27]
[27, 6, 218, 77, 27, 6]
[27, 6, 218, 77, 27, 6, 218]
[27, 6, 218, 77, 27, 6, 218, 77]
[27, 6, 218, 77, 27, 6, 218, 77, 21]
[27, 6]
[27, 6, 218]
[27, 6, 218, 77]
[27, 6, 218, 77, 27]
[27, 6, 218, 77, 27, 6]
[27, 6, 218, 77, 27, 6, 218]
[27, 6, 218, 77, 27, 6, 218, 77]
[27, 6, 218, 77, 27, 6, 218, 77, 21]
[27, 6, 218, 77, 27, 6, 218, 77, 21, 11]
[27, 6]
[27, 6, 218]
[27, 6, 218, 77]
[27, 6, 218, 77, 27]
[27, 6, 218, 77, 27, 6]
[27, 6, 218, 77, 27, 6, 218]
[27, 6, 218, 77, 27, 6, 218, 77]
[27, 6, 218, 77, 27, 6, 218, 77, 21]
[27, 6, 218, 77, 27, 6, 218, 77, 21, 11]
[27, 6, 218, 77, 27, 6, 218, 77, 21, 11, 683]
[27, 6, 218, 77, 27, 6, 218, 77, 21, 11, 683, 129]
[27, 6, 218, 77, 27, 6, 218, 77, 21, 11, 683, 129, 8]
[27, 6, 218, 77, 27, 6, 218, 77, 21, 11, 683, 129, 8, 158]
[27, 6, 218, 77, 27, 6, 218, 77, 21, 11, 683, 129, 8, 158, 243]
[27, 6, 218, 77, 27, 

## 3.1. Padding

We can see our lines of words are not consistent in length, let's transform them to have the same length.

In [45]:
max_length = 20 #let's get the maximum of words per line equal to 20
sequences = pad_sequences(datalist, maxlen = max_length, padding = 'pre')
sequences

array([[  0,   0,   0, ...,   0, 254,  21],
       [  0,   0,   0, ..., 254,  21, 219],
       [  0,   0,   0, ...,   0, 117,   8],
       ...,
       [  0,   0,   0, ...,  17, 198,  17],
       [  0,   0,   0, ..., 198,  17, 198],
       [  0,   0,   0, ...,  17, 198,   6]], dtype=int32)

In [46]:
sequences.shape

(14231, 20)

In [47]:
#let's set the last column of X as our target
y = sequences[:,-1].copy()
X = sequences[:,:-1].copy()

In [48]:
y

array([ 21, 219,   8, ...,  17, 198,   6], dtype=int32)

In [49]:
X

array([[  0,   0,   0, ...,   0,   0, 254],
       [  0,   0,   0, ...,   0, 254,  21],
       [  0,   0,   0, ...,   0,   0, 117],
       ...,
       [  0,   0,   0, ..., 198,  17, 198],
       [  0,   0,   0, ...,  17, 198,  17],
       [  0,   0,   0, ..., 198,  17, 198]], dtype=int32)

In [50]:
#Let's do onehot encoding for y

y = to_categorical(y,num_classes = vocab_size)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [51]:
seq_length = X.shape[1]

# 4. Build and train the model

In [52]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))

model.add(LSTM(100, return_sequences= True))
model.add(LSTM(100))
model.add(Dense(100,activation= 'relu'))
model.add(Dense(vocab_size,activation = 'softmax'))

In [53]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 19, 50)            69800     
_________________________________________________________________
lstm_2 (LSTM)                (None, 19, 100)           60400     
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 1396)              140996    
Total params: 361,696
Trainable params: 361,696
Non-trainable params: 0
_________________________________________________________________


In [55]:
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam', metrics = ['accuracy'])
model.fit(X,y,batch_size = 32,epochs = 100)

Epoch 1/100
445/445 [==============================] - 6s 7ms/step - loss: 1.6764 - accuracy: 0.5873
Epoch 2/100
445/445 [==============================] - 3s 8ms/step - loss: 1.5790 - accuracy: 0.6038
Epoch 3/100
445/445 [==============================] - 3s 7ms/step - loss: 1.5400 - accuracy: 0.6152
Epoch 4/100
445/445 [==============================] - 3s 7ms/step - loss: 1.5138 - accuracy: 0.6223
Epoch 5/100
445/445 [==============================] - 3s 7ms/step - loss: 1.4882 - accuracy: 0.6254
Epoch 6/100
445/445 [==============================] - 3s 7ms/step - loss: 1.4594 - accuracy: 0.6350
Epoch 7/100
445/445 [==============================] - 3s 7ms/step - loss: 1.4368 - accuracy: 0.6378
Epoch 8/100
445/445 [==============================] - 3s 7ms/step - loss: 1.4087 - accuracy: 0.6413
Epoch 9/100
445/445 [==============================] - 3s 7ms/step - loss: 1.3876 - accuracy: 0.6490
Epoch 10/100
445/445 [==============================] - 3s 7ms/step - loss: 1.3731 - accura

# 5. Poetry genetations

In [58]:
poetry_length = 10

def generate_poetry(seed_text, n_lines):

  for i in range(n_lines):
    text = []
    for _ in range(poetry_length):
      encoded = token.texts_to_sequences([seed_text])
      encoded = pad_sequences(encoded, maxlen= max_length, padding = 'pre')

      y_pred = np.argmax(model.predict(encoded),axis = -1)

      predicted_word = ""

      for word,index in token.word_index.items():
        if index == y_pred:
          predicted_word = word
          break

      seed_text = seed_text+ " " + predicted_word
      text.append(predicted_word)

    seed_text = text[-1]
    text = " ".join(text)
    print(text)

In [60]:
seed_text = 'love'
generate_poetry(seed_text,10)

to you yeah and i can tell by the way
i'm feeling sun babe when i'm in your eyes they
keep me thinking that we almost had it all the
scars of your love remind me of all and outside
it crumbles when it hurts instead you know when to
tell you i'm giving you to the way without my
god this reminds in your name your name i set
final back to life to understand just don't want you
know i'm never gonna leave you no go ahead go
head and sell me out and i'll lay your ship


In [61]:
def _remove_accented_chars(x):
	x = unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore')
	return x

In [64]:
x = 'bằng chứng hiển nhiên và sự thật toàn bích'
import unicodedata
_remove_accented_chars(x)

'bang chung hien nhien va su that toan bich'